In [1]:
!pip install pandas transformers
!pip install openpyxl
!pip install torch
!pip install xlrd

In [3]:
import pandas as pd
import torch
import os

# Read the Excel file
notebook_path = os.path.abspath("AI ML POC.ipynb")
file_path = os.path.join(os.path.dirname(notebook_path), "Dataset_Mapped.xlsx")
#file_path = r"Dataset_Mapped.xlsx"
data = pd.read_excel(file_path,engine='openpyxl')

# Create mapping dictionary dynamically
mapping_dictionary = {}

for index, row in data.iterrows():
    key = (row['Question'], row['SelectedOptionId'], row['OptionText'])
    value = (row['FormCategoryId'], row['Category Name'])
    mapping_dictionary[key] = value


In [4]:
from transformers import BertTokenizer, BertForSequenceClassification

# Tokenize the data
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
texts = data['Question'].tolist()
encoded_inputs = tokenizer(texts, padding=True, truncation=True, return_tensors='pt')

# Display the tokens for the first input text
first_input_tokens = tokenizer.convert_ids_to_tokens(encoded_inputs['input_ids'][0])
print("Tokens:", first_input_tokens)

# Load pre-trained classification model
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=len(mapping_dictionary))
model.eval()


Downloading:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Tokens: ['[CLS]', 'did', 'you', 'sell', ',', 'exchange', ',', 'or', 'purchase', 'any', 'other', 'real', 'estate', 'during', 'the', 'year', '?', '[SEP]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '

Downloading:   0%|          | 0.00/420M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [ ]:
# Predict action category for each response
with torch.no_grad():
    outputs = model(**encoded_inputs)
    logits = outputs.logits
    predicted_labels = torch.argmax(logits, dim=1).tolist()
    
# Map predicted labels to action categories
predicted_categories = [mapping_dictionary[(texts[i], 1, 'Yes')][1] for i in range(len(texts))]

# Display predicted categories
for text, category in zip(texts, predicted_categories):
    print(f"Question: {text} \t Predicted Category: {category}")